In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [40]:
import glob
import random 
import time

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from torch.autograd import Variable
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

In [46]:
transform = transforms.Compose([transforms.ToTensor()])

In [47]:
#dataset = datasets.ImageFolder('./rps-cv-images/', transform=transform) # This is all data load at once

In [55]:
train = datasets.ImageFolder('./data/train/', transform=transform)# This is train only
test = datasets.ImageFolder('./data/test/', transform=transform)# This is test only

trainloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

In [59]:
# Get one batch
images, labels = next(iter(trainloader))
print(images,labels)

tensor([[[[0.2000, 0.1922, 0.1922,  ..., 0.2078, 0.2078, 0.2157],
          [0.1647, 0.1686, 0.1647,  ..., 0.2039, 0.2039, 0.2039],
          [0.1725, 0.1686, 0.1725,  ..., 0.2039, 0.2078, 0.2039],
          ...,
          [0.1725, 0.1765, 0.1804,  ..., 0.2275, 0.2314, 0.2392],
          [0.1725, 0.1725, 0.1765,  ..., 0.2314, 0.2275, 0.2392],
          [0.1686, 0.1725, 0.1725,  ..., 0.2275, 0.2275, 0.2353]],

         [[0.6039, 0.6078, 0.6078,  ..., 0.6510, 0.6510, 0.6510],
          [0.6157, 0.6196, 0.6157,  ..., 0.6588, 0.6588, 0.6588],
          [0.6235, 0.6196, 0.6235,  ..., 0.6588, 0.6627, 0.6588],
          ...,
          [0.6510, 0.6549, 0.6588,  ..., 0.6980, 0.7059, 0.6980],
          [0.6510, 0.6510, 0.6549,  ..., 0.7059, 0.6980, 0.6980],
          [0.6471, 0.6510, 0.6510,  ..., 0.6980, 0.6980, 0.6941]],

         [[0.2980, 0.2980, 0.2980,  ..., 0.3294, 0.3294, 0.3373],
          [0.3020, 0.3059, 0.3020,  ..., 0.3294, 0.3294, 0.3373],
          [0.3098, 0.3059, 0.3098,  ..., 0

In [60]:
images, labels = next(iter(testloader))
print(images,labels)

tensor([[[[0.1451, 0.1412, 0.1333,  ..., 0.1569, 0.1451, 0.1333],
          [0.1294, 0.1373, 0.1255,  ..., 0.1412, 0.1294, 0.1255],
          [0.1294, 0.1294, 0.1255,  ..., 0.1412, 0.1412, 0.1294],
          ...,
          [0.1294, 0.1333, 0.1333,  ..., 0.5020, 0.4627, 0.4314],
          [0.1333, 0.1333, 0.1373,  ..., 0.4706, 0.4667, 0.4863],
          [0.1333, 0.1373, 0.1333,  ..., 0.4980, 0.4510, 0.5059]],

         [[0.4588, 0.4667, 0.4588,  ..., 0.4745, 0.4588, 0.4588],
          [0.4667, 0.4706, 0.4627,  ..., 0.4745, 0.4667, 0.4627],
          [0.4667, 0.4667, 0.4627,  ..., 0.4745, 0.4745, 0.4667],
          ...,
          [0.4784, 0.4824, 0.4824,  ..., 0.3647, 0.3216, 0.2902],
          [0.4824, 0.4824, 0.4863,  ..., 0.3294, 0.3255, 0.3451],
          [0.4824, 0.4863, 0.4784,  ..., 0.3608, 0.3137, 0.3765]],

         [[0.1569, 0.1608, 0.1529,  ..., 0.1608, 0.1490, 0.1451],
          [0.1490, 0.1529, 0.1451,  ..., 0.1569, 0.1490, 0.1451],
          [0.1490, 0.1490, 0.1451,  ..., 0